In [47]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

# Getting real estate listings


## Zumber

In [171]:
def Zumber(content): 
    soup = BeautifulSoup(content, "html.parser")
    PostalCode =  [x.text.replace(',', '').strip() for x in soup.find_all("span", class_="css-wmtwp4")]
    rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
    bed_bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]

    bed,bath = [x.split('•')[0] for x in bed_bath],  [x.split('•')[1] for x in bed_bath]


    dict = {'PostalCode':PostalCode,'Monthly': rent,  'Number of beds': bed, 'Number of baths': bath}
    df= pd.DataFrame(dict)
    df.loc[df['Monthly'].str.contains('%'), 'Monthly'] = df['Monthly'].str[:6]
    
    df['Monthly'] = df['Monthly'].apply(lambda x:x.replace('$','').replace(',','').replace("+",'').strip())
    df['Monthly']= df['Monthly'].replace({'K':'*1e3'}, regex=True).map(pd.eval)
    return df

### Studios

In [141]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/studios"
requestURL = requests.get(URL)
content = requestURL.content

In [172]:
Zumber(content)

,PostalCode,Monthly,Number of beds,Number of baths
0,V6G 1P5,1900,Studio,1 Bath
1,V6E 1Z9,1625,Studio,1 Bath
2,V6E 1Z5,2000,Studio,1 Bath
3,V6E 1L7,2369,Studio,1 Bath
4,V6E 2B8,2268,Studio,1 Bath
5,V6G 1X7,2570,Studio,1 Bath
6,V6G 1L3,2650,Studio,1 Bath
7,V6G 1G6,2480,Studio,1 Bath
8,V6G 1W1,2495,Studio,1 Bath
9,V6E 1R9,3500,Studio,1 Bath


### 1 bed

In [142]:
URL2 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/1-beds"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

In [173]:
Zumber(content2)

,PostalCode,Monthly,Number of beds,Number of baths
0,V6G 2C4,3875,1 Bed,1 Bath
1,V6G 1P5,2450,1 Bed,1 Bath
2,V6G 0C6,3450,1 Bed,1 Bath
3,V6E 1N5,2600,1 Bed,1 Bath
4,V6E 1R7,2300,1 Bed,1 Bath
5,V6G 1G2,2375,1 Bed,1 Bath
6,V6G 1W2,2680,1 Bed,1 Bath
7,V6G 1E6,2400,1 Bed,1 Bath
8,V6G 2P5,2200,1 Bed,1 - 2 Baths
9,V6E 1L4,2750,1 Bed,1 Bath


## 2 Beds

In [143]:
URL3 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/2-beds"
requestURL3 = requests.get(URL3)
content3 = requestURL3.content


In [144]:
Zumber(content3)

,Monthly,Number of beds,Number of baths
0,4500.0,2 Beds,2 Baths
1,4100.0,2 Beds,1 - 2 Baths
2,4300.0,2 Beds,1 - 2 Baths
3,3799.0,2 Beds,1 Bath
4,3500.0,2 Beds,1 - 2 Baths
5,3400.0,2 Beds,1 Bath
6,3300.0,2 Beds,1 - 2 Baths
7,7800.0,2 Beds,2 Baths
8,2650.0,2 Beds,1 Bath
9,4900.0,2 Beds,2 Baths


### 3 beds

In [ ]:
URL4 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/3-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content


In [ ]:
Zumber(content4)

## Kijiji

In [ ]:
def kijiji(content): 
    soup = BeautifulSoup(content, "lxml")
    realEstate = soup.findAll("div", class_="container-results large-images")[1]
    realEstate2 = [x.find('div', class_="info") for x in realEstate.findAll("div", attrs={"data-listing-id":True})]

    realEstateLinks = [x.find('div', class_=["title"]) for x in realEstate2]
    realEstateLinks1 = [x.find('a', attrs={"href":True}) for x in realEstate2]
    realEstateLinks = ["https://www.kijiji.ca"+ x['href'] for x in realEstateLinks1]

    dfKijiji = pd.DataFrame(columns=['Monthly', 'Number of baths'])

    for i in range(len(realEstateLinks)):
        requestsIterate = requests.get(realEstateLinks[i])
        soup3 = BeautifulSoup(requestsIterate.text, "html.parser")


        for x in soup3.find_all('div', class_=['realEstateTitle-1440881021']):
            monthlyNum = x.find('span')['content']
            bathroomNum = x.findAll('span', class_="noLabelValue-3861810455")[-1].text

            dfKijiji = dfKijiji.append({'Monthly': monthlyNum, 'Number of baths': bathroomNum}, ignore_index=True)
    return dfKijiji
    



### Studios

In [ ]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/bachelor+studio/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

In [ ]:
kijiji(content2)

### 1 bedroom

In [ ]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/1+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
contentOneBedRoom = requestURL2.content

In [ ]:
kijiji(contentOneBedRoom)

### 2 bed

In [ ]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/2+bedrooms-apartment/c37l1700292a27949001a29276001"
requestURL2 = requests.get(URL2)
contentTwoBedRoom = requestURL2.content

In [ ]:
kijiji(contentTwoBedRoom)

### 3 bed

In [ ]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/3+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
contentThreeBedRoom = requestURL2.content

In [ ]:
kijiji(contentThreeBedRoom)

## Pad Mapper

In [ ]:
def padMapper(content): 
    soupPad = BeautifulSoup(content, "html.parser").find('body')

    realEstate = soupPad.findAll("div", class_="ListItemMobile_text__3KFu_")
    realEstate2 = [x.text.strip() for x in realEstate]
    return realEstate2

### Studio 

In [ ]:
URL3 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/studios"
requestURL3 = requests.get(URL3)
content3 = requestURL3.content

In [ ]:
padMapper(content3)

### 1 bathrooms

In [ ]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/1-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

### 2 bathrooms

In [ ]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/2-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

In [ ]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/3-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

## Rent cafe

In [131]:
def rentCafe(content): 
    soupCafe = BeautifulSoup(content)
    realEstate = soupCafe.findAll("li", class_="data-rent")
    realEstate2 = [x.text.strip() for x in realEstate]
    return realEstate2
    

In [132]:
URLCafe1 = "https://www.rentcafe.com/studio-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$2,025', '$2,240']

In [133]:
URLCafe1 = "https://www.rentcafe.com/1-bedroom-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$2,350+',
 '$2,175+',
 '$2,950',
 '$2,800',
 '$2,700',
 '$2,500',
 '$2,275',
 '$2,270+',
 '$2,100']

In [134]:
URLCafe1 = "https://www.rentcafe.com/2-bedroom-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$3,650',
 '$3,050',
 '$4,500',
 '$3,800',
 '$2,650',
 '$3,250+',
 '$2,800',
 '$2,950+',
 '$2,100']

In [135]:
URLCafe1 = "https://www.rentcafe.com/3-bedroom-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$6,500', '$4,200', '$12,000', '$2,850', '$4,050', '$3,700']